Created 5.19.20

Authors: Margot Wagner, Sam Russman

COGS 260 Neural Data Analysis

In [191]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
from math import ceil

%matplotlib inline

### Load dataset

In [192]:
def load_data(filename, transpose=False):
    '''
    Load in data and return values
    
    param:   filename (str) - filename/path to data
             transpose - transposes data if true (default False)
    return:  data - data values (n_samples, n_features) 
    '''
    data = pd.read_csv(filename, header=None)
    
    # needs to be samples x features
    if transpose:    
        data = data.transpose()
        
    return data.values

In [193]:
def norm_data(data):
    '''
    Normalize data to z-values (0 mean and 1 std dev)
    
    param:   data - data values (n_samples, n_features)           
    return:  data - normalized data
    '''        
    means = data.mean(axis=0)    # mean for each feature
    stdevs = data.std(axis=0)    # std dev for each feature
    data = (data - means) / stdevs    # normalized data (Z-score)
    return data

### PCA 

In [194]:
from sklearn.decomposition import PCA
def run_pca(data):
    '''
    Does PCA on normalized data set. Can optionally set fewer components
    
    param: data - normalized data (n_samples, n_features)
    return: 
        data_pc: data transformed onto princinpal components
        components:  principal axes in feature space, array, shape (n_components, n_features)
        weights: percentage of variance explained by each of the selected components. array, shape (n_components,)
    '''
    # create PCA model
    pca = PCA() 
    
    # fit model to data
    data_pc = pca.fit(data)  
    
    # obtain components and components' weights
    components = pca.components_
    weights = pca.explained_variance_ratio_
    
    return data_pc, components, weights, pca

In [196]:
def cum_var_plot(weights, desired_var, plot=False):
    '''
    Cumulative variance plot (number of components vs cumulative variance captured) with calculated number 
    of PCs required to get to a certain desired variance explained
    
    params:
        weights: percentage of variance explained by each of the selected components. array, shape (n_components,)
        desired_var:  percent variance to find number of PCs for
    return
        pcs_req:   pcs required to captured at least desired variance
        captured_var   exact variance captured by pcs_req
    
    '''
    INDEX_SHIFT = 1
    # cumulative variance captured
    cum_var = np.cumsum(weights) 
    
    # find pcs req to get desired variance
    pcs_req = int(round(np.min(np.where(cum_var > desired_var)) + 1)) 
    
    # actual variance captured
    captured_var = cum_var[pcs_req-INDEX_SHIFT]
    
    # plotting
    if plot:
        plt.figure()
        plt.plot(range(INDEX_SHIFT,len(cum_var)+INDEX_SHIFT), cum_var)
        plt.axvline(x=pcs_req, ymin=0, ymax=1, color='k', linestyle='--')
        plt.xlabel('Number of components')
        plt.ylabel('Cumulative variance captured')
        plt.title('Cumulative Variance Captured by Principal Components')
    
    return pcs_req, captured_var

In [197]:
def biplot(data, pca, plot=False):
    '''
    Creates biplot for data mapping data onto top 2 or 3 principal components
    
    params
        data - normalized data, array shape (n_samples, n_features)
        pca - pca model
        
    return
        top_pcs[:,PC1_IDX] - pcs captured by top PC
        top_var[PC1_IDX] - variance captured by top PC
    '''
    PC1_IDX = 0    # index of first PC
    PC2_IDX = 1    # index of second PC
    DIM = 2        # dimensions to plot 
    
    # top 2 pc's and their variance explained
    top_pcs = pca.transform(data)[:,:DIM]
    top_var = pca.explained_variance_ratio_[:DIM]
    
    # plotting
    if plot:
        plt.figure()
        plt.scatter(top_pcs[:,PC1_IDX], top_pcs[:,PC2_IDX])
        plt.xlabel('PC1 ({:.1%})'.format(top_var[PC1_IDX]))
        plt.ylabel('PC2 ({:.1%})'.format(top_var[PC2_IDX]))
        plt.axvline(x=0, ymin=np.min(top_pcs[:,PC2_IDX]), ymax=np.max(top_pcs[:,PC2_IDX]), color='k', ls='--')
        plt.axhline(y=0, xmin=np.min(top_pcs[:,PC1_IDX]), xmax=np.max(top_pcs[:,PC1_IDX]), color='k', ls='--')
    
    return top_pcs[:,PC1_IDX], top_var[PC1_IDX]

# LDA Classification

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_recall_curve
def lda(data):
    '''
    Linear discriminant analysis (LDA) classifier for input data into 3 stimulus conditions (left, right, no stimulus)
    
    param:
        data - normalized and potentially transformed to pcs
    
    '''
    
    N_TRIALS = 21
    labels = np.array(['left', 'right', 'none'])
    labels = np.repeat(labels, N_TRIALS)
    
    # TRANSFORM DATA TO LOWER DIM
    # Automatically sets test as 0.25
    # Random state so that it's the same every run
    X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state = 0)

    # train LDA classifier
    # look into using LDA for dimensionality reduction??
    lda_model = LinearDiscriminantAnalysis.fit(X_train , y_train)
    y_predict = lda_model.predict(X_test)
    
    # model accuracy for X_test
    lda_acc = 100*accuracy_score(y_test, y_predict)
    print('Accuracy:',round(lda_acc,2),'%')
    print(classification_report(y_test, y_predict,target_names=['Left', 'Right', 'None']))
    
    # creating a confusion matrix
    cm = confusion_matrix(y_test, y_predict)



### Receiver Operating Characteristic (ROC)

ROC curves have true positive (predicted positive and actually positive) rate on the Y axis and false positive (predicted positive, actually negative) on the X axis. THe top left corner for the plot is the "ideal" point with a false positive rate of zero and a true positive rate of one.  
  
In order to extend ROC curve and ROC area to multi-label classification, it is necessary to binarize the output (one vs all). One ROC curve can be drawn per label and/or one can draw a ROC curve by considering each element of the label indicator matrix as a binary prediction (micro-averaging).  
  
Another evaluation for multi-label classification is macro-averaging, which gives equal weight to the classification of each label.  
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#plot-roc-curves-for-the-multilabel-problem

In [ ]:
def roc_curve():
    #https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226
    #https://stackoverflow.com/questions/56090541/how-to-plot-precision-and-recall-of-multiclass-classifier/56092736
    #https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#in-multi-label-settings
    
    # Precision-Recall curve

### TODO: Add regularization step

### TODO: Add bootstrapping?

In [211]:
for filename in glob.glob('*60mA*'):
    print((filename.split('_')[1] + '_trial_' + filename.split('_')[5]).split('.')[0])
    print(filename)
    #print(filename.split('processed_')[1].split('.')[0])
    #trials.append(filename.split('processed_')[1].split('.')[0])

l_trial_1
s_l_60mA_processed_trial_1.csv
l_trial_2
s_l_60mA_processed_trial_2.csv
l_trial_3
s_l_60mA_processed_trial_3.csv
l_trial_4
s_l_60mA_processed_trial_4.csv
l_trial_5
s_l_60mA_processed_trial_5.csv
l_trial_6
s_l_60mA_processed_trial_6.csv
l_trial_7
s_l_60mA_processed_trial_7.csv
l_trial_8
s_l_60mA_processed_trial_8.csv
l_trial_9
s_l_60mA_processed_trial_9.csv
l_trial_10
s_l_60mA_processed_trial_10.csv
l_trial_11
s_l_60mA_processed_trial_11.csv
l_trial_12
s_l_60mA_processed_trial_12.csv
l_trial_13
s_l_60mA_processed_trial_13.csv
l_trial_14
s_l_60mA_processed_trial_14.csv
l_trial_15
s_l_60mA_processed_trial_15.csv
l_trial_16
s_l_60mA_processed_trial_16.csv
l_trial_17
s_l_60mA_processed_trial_17.csv
l_trial_18
s_l_60mA_processed_trial_18.csv
l_trial_19
s_l_60mA_processed_trial_19.csv
l_trial_20
s_l_60mA_processed_trial_20.csv
l_trial_21
s_l_60mA_processed_trial_21.csv
r_trial_1
s_r_60mA_processed_trial_1.csv
r_trial_2
s_r_60mA_processed_trial_2.csv
r_trial_3
s_r_60mA_processed_tria

# Data Analysis

In [234]:
''' Mounted shared Google Drive for data
Data is recordings from 360 good channels x time
Data should be samples x features (time x features)
360 features, 51 samples. '''
'''21 right, 21 left, 20 baseline'''
N_SAMPLES = 62
N_TIMEPOINTS = 51
N_EXTRA_DATA = 2

'''
Output is top pc for all time points (0:50), the variance explained by
top output(51), and the total number of pc's required for 80% variance
explained by number of samples (21 left, 21 right)'''
output = np.zeros([N_TIMEPOINTS + N_EXTRA_DATA, N_SAMPLES])

# 60mA stim and baseline
os.chdir('/Volumes/GoogleDrive/Shared drives/COGS 260 Project/Data/fragments3')
files = glob.glob('*60mA*') + glob.glob('baseline*')

trial_names = []    # initialize list of trials
index = 0      # initialize index

# Run PCA on all trials
for filename in files:
    if '60' in filename:
        trial_names.append((filename.split('_')[1] + '_trial_' 
                            + filename.split('_')[5]).split('.')[0])
    else:
        trial_names.append(filename.split('.')[0])
           
    # Initialize trial results
    trial_results = np.zeros([N_TIMEPOINTS + N_EXTRA_DATA])
    
    # Load and normalize data
    data = load_data(filename, transpose=True)
    data = norm_data(data)
    
    # Run pca
    data_pc, components, weights, pca = run_pca(data)
    
    # further analysis
    # plotting options turned off
    pcs_req, captured_var = cum_var_plot(weights, 0.8)
    top_pc, top_var = biplot(data, pca)

    # Organize results for single trial
    trial_results[:N_TIMEPOINTS] = top_pc
    trial_results[N_TIMEPOINTS] = top_var
    trial_results[N_TIMEPOINTS + 1] = pcs_req

    # Add to all trial output
    output[:, index] = trial_results

    # incremement index
    index += 1
    

In [235]:
pd.DataFrame(output, columns=trial_names)

,l_trial_1,l_trial_2,l_trial_3,l_trial_4,l_trial_5,l_trial_6,l_trial_7,l_trial_8,l_trial_9,l_trial_10,...,baseline_trial_8,baseline_trial_19,baseline_trial_3,baseline_trial_5,baseline_trial_9,baseline_trial_17,baseline_trial_13,baseline_trial_12,baseline_trial_11,baseline_trial_10
0,-7.525431,-12.024689,5.333690,-15.201771,-13.522962,-3.214963,-8.840905,-5.616349,2.267068,18.009457,...,12.210262,9.791788,17.333947,-17.486582,14.212095,-7.116200,15.074694,16.087647,14.377524,-16.764211
1,-8.982833,-11.488158,-3.604749,-23.319618,-14.633243,-5.781539,-7.563470,-6.236100,-1.745985,10.371672,...,11.514328,6.475937,17.447892,-17.363630,17.285204,-10.402046,13.332826,16.327272,14.431303,-16.895334
2,-11.141523,-12.103970,-6.505610,31.331777,-13.598860,-5.007512,-9.575792,-7.779647,-4.553653,7.666959,...,12.714188,1.441351,15.536486,-16.991974,18.033778,-11.204158,12.423728,17.165572,12.651255,-14.712830
3,-10.242713,-13.468507,26.830914,78.790780,-13.752685,-1.185144,-11.357273,-5.609592,-3.901071,35.034961,...,15.353452,2.356066,15.006534,-18.191082,17.730497,-11.128078,12.714966,17.558480,13.502880,-13.360411
4,-7.571089,-9.937053,53.651628,39.594840,-16.090566,0.490573,-9.878586,-1.082359,-3.505413,56.125345,...,16.231822,6.801387,14.792935,-18.481937,17.409206,-11.260803,12.710222,17.749116,15.644306,-13.601264
5,-7.812236,-4.419489,27.917408,-15.595708,-16.497828,-3.240420,-7.503044,-1.525361,-4.285373,32.900691,...,14.246875,7.922877,13.446857,-16.306739,17.838645,-9.858422,11.665634,19.284194,13.183241,-13.263286
6,-9.263641,-5.989432,-6.248622,-13.742040,-14.131065,-7.423902,-7.711403,-5.999808,-4.011297,3.944421,...,10.854264,5.599191,12.390258,-12.438501,18.677284,-8.680475,10.379040,20.867034,9.279631,-11.391396
7,-7.912177,-11.697593,-5.481162,1.329512,-12.181476,-6.163017,-9.041615,-7.321664,-2.634631,4.835585,...,8.876808,2.879411,11.815837,-6.917168,16.561989,-10.653080,10.935372,20.291682,10.618789,-9.009207
8,-6.508985,-12.558046,2.471332,-5.076013,-11.405892,-3.225754,-7.223392,-4.705108,-1.029203,10.861139,...,10.178175,3.745349,10.991884,-2.347654,11.648230,-12.256124,12.754526,19.040447,12.831420,-6.913613
9,-7.790424,-10.514532,-2.436373,-9.288329,-10.825628,-2.228198,-3.899352,-2.068588,0.446678,6.495142,...,12.798802,10.546004,10.670653,-3.661238,8.619640,-9.059787,11.697442,18.514415,9.901065,-5.559305
